In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

# Load the CoLA dataset
cola_dataset = load_dataset("glue", "cola")

# Accessing the train, validation, and test sets
train_data = cola_dataset['train']
test_data = cola_dataset['validation']
# test_data = cola_dataset['test']

# Split the train data into new train and development sets
train_dev_split = cola_dataset['train'].train_test_split(test_size=0.1, seed=42)

# New training and development sets
train_data = train_dev_split['train']
dev_data = train_dev_split['test']



In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer
from torch.optim import AdamW
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

class CoLADataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data=data
        self.tokenizer=tokenizer
        self.max_length=max_length
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        item = self.data[idx]
        sentence = item['sentence']
        label = item['label']

        encoding = self.tokenizer(
            sentence,
            return_tensors='pt',
            max_length=self.max_length,
            padding='max_length',
            truncation=True
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
class BertClassifier(nn.Module):
    def __init__(self, model_name, num_classes=2):
        super(BertClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        logits = self.fc(pooled_output)
        return logits

In [ ]:
train_dataset = CoLADataset(train_data, tokenizer)
dev_dataset = CoLADataset(dev_data, tokenizer)
test_dataset = CoLADataset(test_data, tokenizer)

BATCH_SIZE = 64

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = BertClassifier(model_name)
model = nn.DataParallel(model)
model.to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

def train(model, data_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    
    for batch in tqdm(data_loader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(data_loader)
    return avg_loss

def evaluate(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            logits = model(input_ids, attention_mask)
            _, predicted = torch.max(logits, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [ ]:
EPOCHS=3

for epoch in range(EPOCHS):
    print(f"\n[Epoch {epoch + 1}/{EPOCHS}]")
    
    train_loss = train(model, train_loader, optimizer, criterion, device)
    print(f"Training Loss: {train_loss:.4f}")
    
    dev_acc = evaluate(model, dev_loader, device)
    print(f"Dev Accuracy: {dev_acc:.2f}%")

print("\n" + "*"*40)
test_acc = evaluate(model, test_loader, device)
print(f"Test Accuracy: {test_acc:.2f}%")
print("*"*40)